Capstone Project

In [47]:
#pip install pandas

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3


Reading in the first .csv file

In [49]:
df_prices = pd.read_csv('../data/car_prices.csv')

Running my EDA function to see what I'm working with.

In [50]:
from tools import eda

eda(df_prices)

Columns: Index(['year', 'make', 'model', 'trim', 'body', 'transmission', 'vin', 'state',
       'condition', 'odometer', 'color', 'interior', 'seller', 'mmr',
       'sellingprice', 'saledate'],
      dtype='object')
Shape: (558837, 16)
Data Types: year              int64
make             object
model            object
trim             object
body             object
transmission     object
vin              object
state            object
condition       float64
odometer        float64
color            object
interior         object
seller           object
mmr             float64
sellingprice    float64
saledate         object
dtype: object
Describe:                 year      condition       odometer            mmr  \
count  558837.000000  547017.000000  558743.000000  558799.000000   
mean     2010.038927      30.672365   68320.017767   13769.377495   
std         3.966864      13.402832   53398.542821    9679.967174   
min      1982.000000       1.000000       1.000000      25.000000  

Removing null values

In [51]:
from tools import remove_null
remove_null(df_prices)

Nulls are gone! year            0
make            0
model           0
trim            0
body            0
transmission    0
vin             0
state           0
condition       0
odometer        0
color           0
interior        0
seller          0
mmr             0
sellingprice    0
saledate        0
dtype: int64


Removing columns that aren't pertinent to the analysis

In [52]:
df_prices = df_prices.drop(columns = ['condition', 'odometer', 'color', 'interior', 'seller', 'saledate'])

Confirming that the unnecessary columns were dropped

In [53]:
df_prices.columns

Index(['year', 'make', 'model', 'trim', 'body', 'transmission', 'vin', 'state',
       'mmr', 'sellingprice'],
      dtype='object')

Checking to see what are the unique values listed in the body column, as this is the column that will be used to join to the next table. Converting the values to lower case and removing the spaces to get rid of duplicate body types.

In [54]:
body_values = df_prices['body']
print(body_values.unique())

['SUV' 'Sedan' 'Convertible' 'Coupe' 'Wagon' 'Hatchback' 'Crew Cab'
 'G Coupe' 'G Sedan' 'Elantra Coupe' 'Genesis Coupe' 'Minivan' 'Van'
 'Double Cab' 'CrewMax Cab' 'Access Cab' 'King Cab' 'CTS Coupe'
 'SuperCrew' 'E-Series Van' 'Extended Cab' 'SuperCab' 'G Convertible'
 'Koup' 'Regular Cab' 'Quad Cab' 'CTS-V Coupe' 'sedan' 'G37 Convertible'
 'Club Cab' 'Xtracab' 'Q60 Convertible' 'CTS Wagon' 'G37 Coupe' 'Mega Cab'
 'Cab Plus 4' 'Q60 Coupe' 'Beetle Convertible' 'TSX Sport Wagon'
 'Promaster Cargo Van' 'Cab Plus' 'GranTurismo Convertible' 'CTS-V Wagon'
 'Ram Van' 'convertible' 'minivan' 'Transit Van' 'van' 'regular-cab' 'suv'
 'g sedan' 'g coupe' 'hatchback' 'king cab' 'supercrew' 'g convertible'
 'coupe' 'crew cab' 'wagon' 'e-series van' 'regular cab' 'quad cab'
 'g37 convertible' 'supercab' 'extended cab' 'crewmax cab' 'double cab'
 'genesis coupe' 'access cab' 'mega cab' 'xtracab' 'beetle convertible'
 'cts coupe' 'koup' 'club cab' 'elantra coupe' 'q60 coupe' 'cts-v coupe'
 'transit 

Defining a function to clean up the body column, classifying each 'body' value into one of three categories and creating a column to store those categories. Coupe keeps getting classified as 'Other' for some reason.

In [ ]:
def body_classification(body):
    """
    Reclassify the body types to combine them into comparable types

    Args:
        Take the various values in the body column and classify them into a broader category

    Returns:
        Each entry in the body column is classified as either a passenger car or light truck, depending on what it is

    """
    
    if body in ['Sedan', 'Convertible', 'CTS-V Coupe', 'Coupe', 'Wagon', 'Hatchback', 'G Coupe', 'G Sedan', 'Elantra Coupe', 'Genesis Coupe', 'CTS Coupe', 'G Convertible', 'Koup', 'CTS-V Coupe', 'G37 Convertible', 'Q60 Convertible', 'CTS Wagon', 'G37 Coupe', 'Q60 Coupe', 'Beetle Convertible', 'TSX Sport Wagon', 'GranTurismo Convertible', 'CTS-V Wagon', 'sedan', 'convertible', 'coupe', 'wagon', 'hatchback', 'g coupe', 'g sedan', 'elantra coupe', 'genesis coupe', 'cts coupe', 'g convertible', 'koup', 'cts-v coupe', 'g37 convertible', 'q60 convertible', 'cts wagon', 'g37 coupe', 'q60 coupe', 'beetle convertible', 'tsx sport wagon', 'granturismo convertible', 'cts-v wagon']:
        return 'Passenger Cars'
    elif body in ['SUV', 'Transit Van', 'Crew Cab', 'Minivan', 'Van', 'Double Cab', 'CrewMax Cab', 'Access Cab', 'King Cab', 'SuperCrew', 'E-Series Van', 'Extended Cab', 'SuperCab', 'Regular Cab', 'Quad Cab', 'Club Cab', 'Xtracab', 'Mega Cab', 'Cab Plus 4', 'Promaster Cargo Van', 'Cab Plus', 'Ram Van', 'suv', 'crew cab', 'minivan', 'van', 'double cab', 'crewmax cab', 'access cab', 'king cab', 'supercrew', 'e-series van', 'extended cab', 'supercab', 'regular cab', 'quad cab', 'club cab', 'xtracab', 'mega cab', 'cab plus 4', 'promaster cargo van', 'cab plus', 'ram van', 'transit van', 'regular-cab']:
        return 'Light Trucks'
    else:
        return 'Other'

df_prices['body_class'] = df_prices['body'].apply(body_classification)

df_prices.head

<bound method NDFrame.head of         year    make                model         trim       body  \
0       2015     Kia              Sorento           LX        SUV   
1       2015     Kia              Sorento           LX        SUV   
2       2014     BMW             3 Series   328i SULEV      Sedan   
3       2015   Volvo                  S60           T5      Sedan   
4       2014     BMW  6 Series Gran Coupe         650i      Sedan   
...      ...     ...                  ...          ...        ...   
558831  2011     BMW             5 Series         528i      Sedan   
558833  2012     Ram                 2500  Power Wagon   Crew Cab   
558834  2012     BMW                   X5    xDrive35d        SUV   
558835  2015  Nissan               Altima        2.5 S      sedan   
558836  2014    Ford                F-150          XLT  SuperCrew   

       transmission                vin state      mmr  sellingprice  \
0         automatic  5xyktca69fg566472    ca  20500.0       21500.0   

Filtering out vehicles sold earlier than 1994 to mesh with my other dataframe

In [56]:
df_prices = df_prices[df_prices['year'] > 1993]
df_prices.describe() #checking the max and min numbers for year to make sure it did what I wanted.

,year,mmr,sellingprice
count,472017.000000,472017.000000,472017.000000
mean,2010.222842,13845.442802,13698.849052
std,3.794989,9529.633266,9610.561942
min,1994.000000,25.000000,1.000000
25%,2008.000000,7450.000000,7200.000000
50%,2012.000000,12350.000000,12200.000000
75%,2013.000000,18300.000000,18200.000000
max,2015.000000,182000.000000,230000.000000


In [61]:
body_type_by_year = pd.pivot_table(
    df_prices,
    index = 'year',
    columns = 'body_class',
    aggfunc = 'size',
    fill_value = 0
).reset_index()

body_type_by_year.columns.name = None
body_type_by_year = body_type_by_year.rename(columns = {
    'Light Trucks': 'Light Trucks Sold',
    'Passenger Cars': 'Passenger Cars Sold'
})

print(body_type_by_year)

    year  Light Trucks Sold  Passenger Cars Sold
0   1994                 43                  243
1   1995                 92                  391
2   1996                114                  448
3   1997                251                  791
4   1998                322                 1142
5   1999                652                 1575
6   2000               1021                 2406
7   2001               1957                 3183
8   2002               3204                 4489
9   2003               4603                 5765
10  2004               6972                 6652
11  2005               8463                 8706
12  2006               9631                12000
13  2007              12013                13365
14  2008              13150                13861
15  2009               7287                10672
16  2010              10416                12200
17  2011              18600                22784
18  2012              27386                59994
19  2013            

Determining the average sale price by year

In [ ]:
# avg_saleprice = df_prices.groupby('year')['sellingprice'].mean().reset_index()
# avg_saleprice.columns = ['year', 'avg_saleprice'] 

In [ ]:
# plt.bar(avg_saleprice['year'], avg_saleprice['avg_saleprice'], color = 'orange', edgecolor = 'black')
# plt.title('Average Selling Price by Year')
# plt.xlabel('Year')
# plt.ylabel('Average Price')
# plt.show()

Figuring out how many of each make of vehicle was sold over the period represented in the dataframe.

In [ ]:
# total_manufac = df_prices['make'].value_counts().reset_index()
# print(total_manufac)

In [ ]:
# plt.bar(total_manufac['make'], total_manufac['count'], color = 'green', edgecolor = 'black')
# plt.title('Total Vehicles Sold by Manufacturer')
# plt.xlabel('Manufacturer')
# plt.ylabel('Vehicles Sold')
# plt.xticks(rotation=45, ha='right')
# plt.show()

Comparing the two vehicle clases that are in the dataframe to see which type sold the most, and making a donut pie chart because it looks like a wheel.

In [ ]:
# trucks_vs_cars = df_prices['body_class'].value_counts()
# print(trucks_vs_cars)

In [ ]:
# values = trucks_vs_cars.values
# labels = trucks_vs_cars.index

# plt.figure(figsize=(4,4))
# plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=180, colors=plt.cm.Paired.colors, wedgeprops={'width':0.6, 'edgecolor':'white'})
# plt.legend(title = 'Body Class', bbox_to_anchor=(1.05,1), loc='best')
# plt.title('Comparision of Body Classes Sold')
# plt.axis('equal')
# plt.show()

In [62]:
#Connecting the database that was already created
conn = sqlite3.connect('../data/final_data.db')
cursor = conn.cursor()

#Adding the data from the dataframe to a table
body_type_by_year.to_sql('vehicle_prices', conn, if_exists='replace', index = False)

#Verifying that the data was added to the table
print(pd.read_sql_query('SELECT * FROM vehicle_prices', conn))

conn.commit()
conn.close()





    year  Light Trucks Sold  Passenger Cars Sold
0   1994                 43                  243
1   1995                 92                  391
2   1996                114                  448
3   1997                251                  791
4   1998                322                 1142
5   1999                652                 1575
6   2000               1021                 2406
7   2001               1957                 3183
8   2002               3204                 4489
9   2003               4603                 5765
10  2004               6972                 6652
11  2005               8463                 8706
12  2006               9631                12000
13  2007              12013                13365
14  2008              13150                13861
15  2009               7287                10672
16  2010              10416                12200
17  2011              18600                22784
18  2012              27386                59994
19  2013            

Exporting the finished dataframe as a .csv file, if I want to.

In [ ]:
#df_prices.to_csv('../data/car_prices_updated.csv', index=False)